In [6]:
pip install openpyxl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.0/250.0 kB 2.8 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [50]:
# Importamos las librerías

import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns
import openpyxl

In [60]:
# Ruta del PDF
df1 = pd.read_excel('/Users/fernandopradagorge/Documents/Documentos - MacBook Pro de Fernando/Universidad/Master Data Science and AI/Nuclio Projects/Projects/Evolución ventas vehículos particulares/Lista detallada de ventas.xlsx')

In [61]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7907 entries, 0 to 7906
Data columns (total 36 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   V                      7906 non-null   object        
 1   Fecha                  7904 non-null   datetime64[ns]
 2   V/A                    7904 non-null   object        
 3   IDV                    7904 non-null   float64       
 4   Origen                 7904 non-null   object        
 5   Referencia             7904 non-null   float64       
 6   Tipo Vehículo          7686 non-null   object        
 7   Matricula              7814 non-null   object        
 8   Marca                  7904 non-null   object        
 9   Modelo                 7904 non-null   object        
 10  Canal                  7904 non-null   object        
 11  Facturación            7905 non-null   float64       
 12  Facturación (media)    7905 non-null   float64       
 13  Imp

In [62]:
df1.head()

,V,Fecha,V/A,IDV,Origen,Referencia,Tipo Vehículo,Matricula,Marca,Modelo,...,Cambio,Kilómetros,Fecha Matric.,Segmento,SubSegmento,Importe Remarketing,Margen I,Margen II,Margen III,Margen IV
0,1,2024-01-18,V,156385.0,VO,31114932.0,MOTOS,6457GSW,PIAGGIO,PIAGGIO VESPA 50CC,...,AUTOMATICO,72940.0,2010-01-18,NaN,NaN,0.00,82.65,82.65,-48.56,-48.56
1,1,2023-03-21,V,156385.0,VO,28637837.0,MOTOS,6457GSW,PIAGGIO,PIAGGIO VESPA 50CC,...,AUTOMATICO,72940.0,2010-01-18,NaN,NaN,0.00,413.22,413.22,476.84,476.84
2,1,2023-06-29,V,159474.0,VO,29507497.0,TURISMOS,5383GTM,FORD,FOCUS,...,MANUAL,124380.0,2010-02-22,Turismos,Segmento C,0.00,991.73,991.73,855.35,855.35
3,1,2024-03-18,V,200955.0,VO,31567110.0,TURISMOS,4134HSR,OPEL,OPEL CORSA,...,MANUAL,106772.0,2013-08-29,Turismos,Segmento B,1347.61,4123.97,4123.97,2861.92,2861.92
4,1,2023-01-16,V,205707.0,VO,28058110.0,TURISMOS,1962HTZ,ALFA,GIULIETTA,...,MANUAL,66511.0,2013-12-18,Turismos,Segmento C,0.00,2300.51,2300.51,1618.87,1618.87


In [63]:
df1.columns

Index(['V', 'Fecha', 'V/A', 'IDV', 'Origen', 'Referencia', 'Tipo Vehículo',
       'Matricula', 'Marca', 'Modelo', 'Canal', 'Facturación',
       'Facturación (media)', 'Importe Venta (media)', 'Margen', 'Financiado',
       'Imp. Financiado', 'Interés', 'Financiera', 'NumeroCuotas',
       'Concesionario', 'Vendedor', 'Comprador', 'Días en Stock', 'Bastidor',
       'Combustible', 'Cambio', 'Kilómetros', 'Fecha Matric.', 'Segmento',
       'SubSegmento', 'Importe Remarketing', 'Margen I', 'Margen II',
       'Margen III', 'Margen IV'],
      dtype='object')

In [64]:
df1 = df1.drop(['Referencia', 'Canal', 'Matricula', 'Financiado', 'Imp. Financiado', 'Interés', 'Financiera', 'NumeroCuotas', 'Vendedor', 'Comprador', 'Bastidor', 'Kilómetros', 'V', 'V/A', 'IDV', 'Margen I', 'Margen II', 'Margen III', 'Margen IV', 'Segmento', 'SubSegmento', 'Fecha Matric.', 'Cambio', 'Días en Stock', 'Importe Remarketing', 'Margen'], axis = 1)

In [65]:
df1.columns

Index(['Fecha', 'Origen', 'Tipo Vehículo', 'Marca', 'Modelo', 'Facturación',
       'Facturación (media)', 'Importe Venta (media)', 'Concesionario',
       'Combustible'],
      dtype='object')

In [66]:
df2 = df1.copy()

In [67]:
df2.columns

Index(['Fecha', 'Origen', 'Tipo Vehículo', 'Marca', 'Modelo', 'Facturación',
       'Facturación (media)', 'Importe Venta (media)', 'Concesionario',
       'Combustible'],
      dtype='object')

In [68]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7907 entries, 0 to 7906
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   Fecha                  7904 non-null   datetime64[ns]
 1   Origen                 7904 non-null   object        
 2   Tipo Vehículo          7686 non-null   object        
 3   Marca                  7904 non-null   object        
 4   Modelo                 7904 non-null   object        
 5   Facturación            7905 non-null   float64       
 6   Facturación (media)    7905 non-null   float64       
 7   Importe Venta (media)  7905 non-null   float64       
 8   Concesionario          7904 non-null   object        
 9   Combustible            7224 non-null   object        
dtypes: datetime64[ns](1), float64(3), object(6)
memory usage: 617.9+ KB


In [70]:
df2.isnull().sum()

Fecha                      3
Origen                     3
Tipo Vehículo            221
Marca                      3
Modelo                     3
Facturación                2
Facturación (media)        2
Importe Venta (media)      2
Concesionario              3
Combustible              683
dtype: int64